# Finetuning d'un LLM à la façon du Conte de Monte Christo

L'objectif de ce projet et de s'entrainer à faire du finetunig de LLM. Pour cela on utilisera la méthode du QLoRA afin de ne pas avoir à entrainer totalement un LLM 'de zéro'.

## Nettoyage du texte pour former un dataset : 

- Supprimer les pages inutiles (Titre, sommaire, notes, etc...) ==> rawText_*.txt
- Ne garder que les dialogues ==> rawText_*txt
- Identifier puis écrire dans un fichier les citations du Comte de Monte Cristo (avec phrase précédente) ==> dataset_CMC.JSONL



In [42]:
# Filtrage des dialogues (to cleanText_*.txt)

import os
import re


for i in range(1, 4):

    nom_fichier = f"media/raw/rawText_{i}.txt"
    with open(nom_fichier, 'r', encoding='utf-8') as f:
        lignes = f.readlines()

    # Filtrer les lignes vides
    lignes_propres = [ligne for ligne in lignes if ligne.strip()]
    
    with open(f"media/raw/cleanBlankLignes_{i}.txt", 'w', encoding='utf-8') as f:
        f.writelines(lignes_propres)
    
    print(f"Fichier {i} nettoyé !")

#  Filtrer les dialogues (contenus dans « »)

for i in range(1, 4):

    nom_fichier = f"media/raw/cleanBlankLignes_{i}.txt"
    with open(nom_fichier, 'r', encoding='utf-8') as f:
        contenu = f.read()

    # Extraire les dialogues entre « et »
    dialogues = re.findall(r'«(.*?)»', contenu, re.DOTALL)

    # Écrire les dialogues extraits dans le fichier
    with open(f"media/raw/cleanText_{i}.txt", 'w', encoding='utf-8') as f:
        for dialogue in dialogues:
            f.write(dialogue.strip() + '\n')

    print(f"Dialogues extraits dans le fichier {i} !")

# Fusion en un seul fichier

with open('media/cleanDataset.txt', 'w', encoding='utf-8') as fichier_final:
    for i in range(1, 4):
        nom_fichier = f"media/raw/cleanText_{i}.txt"
        with open(nom_fichier, 'r', encoding='utf-8') as f:
            contenu = f.read()
            fichier_final.write(contenu + '\n')


Fichier 1 nettoyé !
Fichier 2 nettoyé !
Fichier 3 nettoyé !
Dialogues extraits dans le fichier 1 !
Dialogues extraits dans le fichier 2 !
Dialogues extraits dans le fichier 3 !


In [5]:
import os
import json
import time
from tqdm import tqdm
from dotenv import load_dotenv
from google import genai
from google.genai import types

# 1. Configuration
load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=API_KEY)

SYSTEM_PROMPT = """Tu es un expert en curation de données littéraires. 
Ta mission : extraire les répliques d'EDMOND DANTÈS (Le Comte).
- Ignore les serviteurs.
- Développe l'output en 3-5 phrases (style XIXe siècle).
- Réponds UNIQUEMENT en JSON (liste d'objets)."""

def run_extraction_with_resume():
    input_path = "media/cleanDataset.txt" 
    output_path = "dantes_dataset_elaborated.jsonl"
    checkpoint_path = "checkpoint_dantes.txt" # Notre marque-page
    chunk_size = 100 

    if not os.path.exists(input_path):
        print(f"❌ Erreur : {input_path} introuvable.")
        return

    # Lecture du fichier source
    with open(input_path, "r", encoding="utf-8") as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]

    # --- LOGIQUE DE REPRISE ---
    start_index = 0
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, "r") as f:
            try:
                start_index = int(f.read().strip())
                print(f"🔄 Reprise détectée à la ligne {start_index}.")
            except:
                print("⚠️ Checkpoint corrompu, on repart de zéro.")

    total_lines = len(lines)
    total_steps = (total_lines - start_index + chunk_size - 1) // chunk_size

    # 2. Boucle d'extraction
    # On commence la boucle à 'start_index'
    with tqdm(total=total_lines, initial=start_index, desc="Curation") as pbar:
        for i in range(start_index, total_lines, chunk_size):
            chunk = lines[i : i + chunk_size]
            prompt_text = "\n".join(chunk)
            
            try:
                response = client.models.generate_content(
                    model="gemini-2.0-flash",
                    config=types.GenerateContentConfig(
                        system_instruction=SYSTEM_PROMPT,
                        response_mime_type="application/json",
                        temperature=0.7
                    ),
                    contents=f"Extraits à traiter :\n{prompt_text}"
                )
                
                # Nettoyage et sauvegarde des données
                raw_text = response.text.strip()
                if raw_text.startswith("```"):
                    raw_text = raw_text.split("json")[-1].split("```")[0].strip()
                
                data = json.loads(raw_text)
                
                if isinstance(data, list) and data:
                    with open(output_path, "a", encoding="utf-8") as out_file:
                        for entry in data:
                            out_file.write(json.dumps(entry, ensure_ascii=False) + "\n")
                
                # --- MISE À JOUR DU CHECKPOINT ---
                # On enregistre l'indice du prochain bloc à traiter
                with open(checkpoint_path, "w") as f_check:
                    f_check.write(str(i + chunk_size))

                pbar.update(len(chunk))
                time.sleep(6)

            except Exception as e:
                tqdm.write(f"⚠️ Erreur lot ligne {i} : {e}")
                time.sleep(10)

    print(f"\n✨ Extraction terminée ou mise en pause. Checkpoint : {checkpoint_path}")

if __name__ == "__main__":
    run_extraction_with_resume()

🔄 Reprise détectée à la ligne 22900.


Curation: 22900it [00:00, ?it/s]


✨ Extraction terminée ou mise en pause. Checkpoint : checkpoint_dantes.txt


In [6]:
# Normalisation du fichier d'entrainement :

import json

def finalize_dataset(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f, \
         open(output_file, 'w', encoding='utf-8') as out:
        
        for line in f:
            data = json.loads(line)
            # On unifie les clés (speaker ou character -> instruction)
            # On utilise le contexte pour fabriquer une consigne
            context = data.get("context", "Le Comte s'exprime avec noblesse.")
            dialogue = data.get("dialogue", "")
            
            # Nouveau format "Instruct"
            new_entry = {
                "instruction": f"Réponds avec le ton du Comte de Monte-Cristo. Contexte : {context}",
                "output": dialogue
            }
            
            out.write(json.dumps(new_entry, ensure_ascii=False) + "\n")

finalize_dataset("dantes_dataset_elaborated.jsonl", "CMC_dataset.jsonl")

In [8]:
import json
import re

def ultra_clean_dataset(input_file, output_file):
    cleaned_data = []
    
    # Liste de mots à supprimer (incises courantes)
    incises = [", dit le comte", ", dit-il", ", s'écria-t-il", ", répondit Dantès", ", reprit Monte-Cristo"]

    with open(input_file, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            output = data.get("output")

            # 1. On ignore les vides ou null
            if not output or output.strip() == "" or output == "null":
                continue
            
            # 2. Nettoyage des incises
            for incise in incises:
                output = output.replace(incise, "")
            
            # 3. Nettoyage des guillemets et espaces
            output = output.replace('«', '').replace('»', '').replace('"', '').strip()
            
            # 4. Mise à jour et sauvegarde
            data["output"] = output
            cleaned_data.append(data)

    with open(output_file, 'w', encoding='utf-8') as out:
        for entry in cleaned_data:
            out.write(json.dumps(entry, ensure_ascii=False) + "\n")
            
    print(f"Nettoyage terminé : {len(cleaned_data)} lignes valides conservées sur 3080.")

ultra_clean_dataset("CMC_dataset.jsonl", "CMC_dataset_FINAL.jsonl")

Nettoyage terminé : 2718 lignes valides conservées sur 3080.
